# Configuración Inicial
Métodos un utils en común en los notebooks

In [12]:
# Imports 
from pyspark.sql import functions as f, SparkSession
from pyspark import SparkContext, SparkConf, SQLContext


In [3]:
class MySQLConnector:
    def __init__(self, spark: SparkSession, connection_properties: dict, url: str):
        self.spark = spark
        self.properties = connection_properties
        self.url = url

    def get_dataframe(self, sql_query: str):        
        df = self.spark.read.jdbc(
            url=self.url,
            table=sql_query,
            properties=self.properties
        )
        return df
    
    def save_db(self, df, tabla):
        df.write.jdbc(
            url=self.url,
            table=tabla,
            mode='append',
            properties=self.properties
        )
        
def create_spark_session(path_jar_driver):    
    conf = SparkConf().set('spark.driver.extraClassPath', path_jar_driver)
    spark_context = SparkContext(conf=conf)
    sql_context = SQLContext(spark_context)
    return sql_context.sparkSession  

def get_dataframe_from_csv(_PATH, _sep):
    return spark.read.load(_PATH, format="csv", sep=_sep, inferSchema="true", header='true')

In [4]:
db_user = 'Estudiante_65_202415'
db_psswd = 'Estudiante_202010409'

connection_properties = {
    "user": db_user,
    "password": db_psswd,
    "driver": "com.mysql.cj.jdbc.Driver"
}

source_db_string_connection = 'jdbc:mysql://157.253.236.120:8080/RaSaTransaccional_ETL'
destination_db_string_connection = f'jdbc:mysql://157.253.236.120:8080/{db_user}'

# Driver de conexion
# LINUX
path_jar_driver = '/opt/mysql/lib/mysql-connector-java-8.0.28.jar'
# WINDOWS
#path_jar_driver = 'C:\Program Files (x86)\MySQL\Connector J 8.0\mysql-connector-java-8.0.28.jar'

In [5]:
spark = create_spark_session(path_jar_driver)

24/11/18 19:30:04 WARN Utils: Your hostname, willp resolves to a loopback address: 127.0.1.1; using 192.168.0.6 instead (on interface enp8s0)
24/11/18 19:30:04 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/11/18 19:30:05 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
/home/willp/anaconda3/lib/python3.11/site-packages/pyspark/sql/context.py:112: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  warnings.warn(


In [6]:
conn_orig = MySQLConnector(spark=spark, connection_properties=connection_properties, url=source_db_string_connection)
conn_dest = MySQLConnector(spark=spark, connection_properties=connection_properties, url=destination_db_string_connection)

# Dimensión Tipo Beneficio

## Extraction

In [7]:
# Se extraen los datos y se muestran
sql_tipo_beneficio = '''(SELECT DISTINCT IdTipoBeneficio_T, Nombre FROM RaSaTransaccional_ETL.FuenteTiposBeneficio_ETL) AS Beneficios'''
tipo_beneficio = conn_orig.get_dataframe(sql_tipo_beneficio)
tipo_beneficio.show()

+-----------------+--------------------+
|IdTipoBeneficio_T|              Nombre|
+-----------------+--------------------+
|                5|Abortion For Whic...|
|               10|   Accidental Dental|
|               20|Adult Frames And ...|
|               25|  Allergy Injections|
|               30|     Allergy Testing|
|               40|          Anesthesia|
|               45|Anesthesia Servic...|
|               50|Applied Behavior ...|
|               60|Autism Spectrum D...|
|               65|Autism Spectrum D...|
|               70|Autism Spectrum D...|
|               75|   Bariatric Surgery|
|               80|Basic Dental Care...|
|               85|Basic Dental Care...|
|               90|Blood And Blood S...|
|               95| Bone Marrow Testing|
|              100|Bone Marrow Trans...|
|              105|        Brain Injury|
|              110|Breast Implant Re...|
|              115|   Cancer Treatments|
+-----------------+--------------------+
only showing top

## Transformation
Se incluye el indice de la bodega de datos y se ordenan las columnas en el dataframe


In [8]:
tipo_beneficio = tipo_beneficio.coalesce(1).withColumn('IdTipoBeneficio_DWH', f.monotonically_increasing_id() + 1)


In [10]:
tipo_beneficio.printSchema()

root
 |-- IdTipoBeneficio_T: integer (nullable = true)
 |-- Nombre: string (nullable = true)
 |-- IdTipoBeneficio_DWH: long (nullable = false)



In [11]:
ben_data = [(0, "Missing", 0)]
ben_columns = ["IdTipoBeneficio_T", "Nombre", "IdTipoBeneficio_DWH"]
dummy_ben = spark.createDataFrame(ben_data, ben_columns)

tipo_beneficio = dummy_ben.union(tipo_beneficio)
tipo_beneficio.show(40)


+-----------------+--------------------+-------------------+
|IdTipoBeneficio_T|              Nombre|IdTipoBeneficio_DWH|
+-----------------+--------------------+-------------------+
|                0|             Missing|                  0|
|                5|Abortion For Whic...|                  1|
|               10|   Accidental Dental|                  2|
|               20|Adult Frames And ...|                  3|
|               25|  Allergy Injections|                  4|
|               30|     Allergy Testing|                  5|
|               40|          Anesthesia|                  6|
|               45|Anesthesia Servic...|                  7|
|               50|Applied Behavior ...|                  8|
|               60|Autism Spectrum D...|                  9|
|               65|Autism Spectrum D...|                 10|
|               70|Autism Spectrum D...|                 11|
|               75|   Bariatric Surgery|                 12|
|               80|Basic

## Load

In [12]:
conn_dest.save_db(tipo_beneficio, 'Rs_TiposBeneficio')

# Dimensión MiniCondiciones Tipo de Servicio

## Extraction

In [18]:
sql_mini_condiciones = '''(
SELECT DISTINCT IdTipoBeneficio_T,
                EstaCubiertaPorSeguro as EstaCubiertoPorSeguro,
                EsEHB,
                TieneLimiteCuantitativo,
                ExcluidoDelDesembolsoMaximoDentroDeLaRed,
                ExcluidoDelDesembolsoMaximoFueraDeLaRed,
                UnidadDelLimite,
                Fecha                 as Annio
FROM RaSaTransaccional_ETL.FuenteTiposBeneficio_ETL
) AS Condiciones'''
mini_condiciones = conn_orig.get_dataframe(sql_mini_condiciones)
mini_condiciones.show()

+-----------------+---------------------+-----+-----------------------+----------------------------------------+---------------------------------------+--------------------+-----+
|IdTipoBeneficio_T|EstaCubiertoPorSeguro|EsEHB|TieneLimiteCuantitativo|ExcluidoDelDesembolsoMaximoDentroDeLaRed|ExcluidoDelDesembolsoMaximoFueraDeLaRed|     UnidadDelLimite|Annio|
+-----------------+---------------------+-----+-----------------------+----------------------------------------+---------------------------------------+--------------------+-----+
|                5|                   No|   No|                     No|                                      No|                                     No|                    | 2017|
|                5|                  Yes|   No|                     No|                                      No|                                    Yes|                    | 2020|
|               10|                  Yes|  Yes|                    Yes|                             

### Transformation

In [19]:
# Se incluye el indice de la bodega de datos y se ordenan las columnas en el dataframe
mini_condiciones = mini_condiciones.coalesce(1).withColumn('IdCondicionesBeneficios_DWH', f.monotonically_increasing_id() + 1)
mini_condiciones.show()

+-----------------+---------------------+-----+-----------------------+----------------------------------------+---------------------------------------+--------------------+-----+---------------------------+
|IdTipoBeneficio_T|EstaCubiertoPorSeguro|EsEHB|TieneLimiteCuantitativo|ExcluidoDelDesembolsoMaximoDentroDeLaRed|ExcluidoDelDesembolsoMaximoFueraDeLaRed|     UnidadDelLimite|Annio|IdCondicionesBeneficios_DWH|
+-----------------+---------------------+-----+-----------------------+----------------------------------------+---------------------------------------+--------------------+-----+---------------------------+
|                5|                   No|   No|                     No|                                      No|                                     No|                    | 2017|                          1|
|                5|                  Yes|   No|                     No|                                      No|                                    Yes|                

In [20]:
# Adicionar Comodin
mini_data = [(0,'No','No','No','No', 'No', "Missing", 0, 0)]
mini_columns = ["IdTipoBeneficio_T", "EstaCubiertoPorSeguro", "EsEHB", "TieneLimiteCuantitativo", "ExcluidoDelDesembolsoMaximoDentroDeLaRed", "ExcluidoDelDesembolsoMaximoFueraDeLaRed", "UnidadDelLimite", "Annio", "IdCondicionesBeneficios_DWH"]
dummy_mini = spark.createDataFrame(mini_data, mini_columns)

mini_condiciones = dummy_mini.union(mini_condiciones)
mini_condiciones.show(3)


+-----------------+---------------------+-----+-----------------------+----------------------------------------+---------------------------------------+---------------+-----+---------------------------+
|IdTipoBeneficio_T|EstaCubiertoPorSeguro|EsEHB|TieneLimiteCuantitativo|ExcluidoDelDesembolsoMaximoDentroDeLaRed|ExcluidoDelDesembolsoMaximoFueraDeLaRed|UnidadDelLimite|Annio|IdCondicionesBeneficios_DWH|
+-----------------+---------------------+-----+-----------------------+----------------------------------------+---------------------------------------+---------------+-----+---------------------------+
|                0|                   No|   No|                     No|                                      No|                                     No|        Missing|    0|                          0|
|                5|                   No|   No|                     No|                                      No|                                     No|               | 2017|              

In [21]:
# Se exploran los datos de las columnas categoricas de tipo 'Yes/No' para verificar que solo existan este tipo de valores
mini_condiciones.groupby('EstaCubiertoPorSeguro').count().orderBy("count", ascending=False).show()
mini_condiciones.groupby('EsEHB').count().orderBy("count", ascending=False).show()
mini_condiciones.groupby('TieneLimiteCuantitativo').count().orderBy("count", ascending=False).show()
mini_condiciones.groupby('ExcluidoDelDesembolsoMaximoDentroDeLaRed').count().orderBy("count", ascending=False).show()
mini_condiciones.groupby('ExcluidoDelDesembolsoMaximoFueraDeLaRed').count().orderBy("count", ascending=False).show()

+---------------------+-----+
|EstaCubiertoPorSeguro|count|
+---------------------+-----+
|                  Yes|  313|
|                   No|   31|
|                False|    2|
+---------------------+-----+

+-----+-----+
|EsEHB|count|
+-----+-----+
|   No|  226|
|  Yes|  103|
| True|   17|
+-----+-----+

+-----------------------+-----+
|TieneLimiteCuantitativo|count|
+-----------------------+-----+
|                     No|  217|
|                    Yes|  119|
|                     Si|   10|
+-----------------------+-----+

+----------------------------------------+-----+
|ExcluidoDelDesembolsoMaximoDentroDeLaRed|count|
+----------------------------------------+-----+
|                                      No|  328|
|                                     Yes|   18|
+----------------------------------------+-----+

+---------------------------------------+-----+
|ExcluidoDelDesembolsoMaximoFueraDeLaRed|count|
+---------------------------------------+-----+
|                         

In [22]:
# Se define una función que transforme los datos 'Si' y 'True' a valores 'Yes'; y los datos 'False' a valores 'No'
def transformar_valores(df, nombre_columna):
    return df.withColumn(
        nombre_columna,
        f.when(f.col(nombre_columna).isin('Si', 'True'), 'Yes') \
         .when(f.col(nombre_columna) == 'False', 'No') \
         .otherwise(f.col(nombre_columna))
    )

# Se especifican ahora las columnas que se deben transformar segun la mini-exploración anterior
columnas_a_transformar = ['EstaCubiertoPorSeguro', 'EsEHB', 'TieneLimiteCuantitativo']

# Se realiza la transformación de las columnas
for col in columnas_a_transformar:
    mini_condiciones = transformar_valores(mini_condiciones, col)
    
# Se inspecciona el dataframe
mini_condiciones.show()

+-----------------+---------------------+-----+-----------------------+----------------------------------------+---------------------------------------+--------------------+-----+---------------------------+
|IdTipoBeneficio_T|EstaCubiertoPorSeguro|EsEHB|TieneLimiteCuantitativo|ExcluidoDelDesembolsoMaximoDentroDeLaRed|ExcluidoDelDesembolsoMaximoFueraDeLaRed|     UnidadDelLimite|Annio|IdCondicionesBeneficios_DWH|
+-----------------+---------------------+-----+-----------------------+----------------------------------------+---------------------------------------+--------------------+-----+---------------------------+
|                0|                   No|   No|                     No|                                      No|                                     No|             Missing|    0|                          0|
|                5|                   No|   No|                     No|                                      No|                                     No|                

In [23]:
# Se exploran nuevamente los datos de las columnas categoricas de tipo 'Yes/No' para verificar que solo existan este tipo de valores
mini_condiciones.groupby('EstaCubiertoPorSeguro').count().orderBy("count", ascending=False).show()
mini_condiciones.groupby('EsEHB').count().orderBy("count", ascending=False).show()
mini_condiciones.groupby('TieneLimiteCuantitativo').count().orderBy("count", ascending=False).show()
mini_condiciones.groupby('ExcluidoDelDesembolsoMaximoDentroDeLaRed').count().orderBy("count", ascending=False).show()
mini_condiciones.groupby('ExcluidoDelDesembolsoMaximoFueraDeLaRed').count().orderBy("count", ascending=False).show()

+---------------------+-----+
|EstaCubiertoPorSeguro|count|
+---------------------+-----+
|                  Yes|  313|
|                   No|   33|
+---------------------+-----+

+-----+-----+
|EsEHB|count|
+-----+-----+
|   No|  226|
|  Yes|  120|
+-----+-----+

+-----------------------+-----+
|TieneLimiteCuantitativo|count|
+-----------------------+-----+
|                     No|  217|
|                    Yes|  129|
+-----------------------+-----+

+----------------------------------------+-----+
|ExcluidoDelDesembolsoMaximoDentroDeLaRed|count|
+----------------------------------------+-----+
|                                      No|  328|
|                                     Yes|   18|
+----------------------------------------+-----+

+---------------------------------------+-----+
|ExcluidoDelDesembolsoMaximoFueraDeLaRed|count|
+---------------------------------------+-----+
|                                     No|  237|
|                                    Yes|  109|
+-----

In [25]:
# Por ultimo se verifica que la columna Annio no contenga nulos y se verifica el tipo de dato
mini_condiciones.groupby('Annio').count().orderBy("count", ascending=False).show()
mini_condiciones.printSchema()

+-----+-----+
|Annio|count|
+-----+-----+
| 2017|  155|
| 2018|   62|
| 2021|   58|
| 2019|   37|
| 2020|   33|
|    0|    1|
+-----+-----+

root
 |-- IdTipoBeneficio_T: long (nullable = true)
 |-- EstaCubiertoPorSeguro: string (nullable = true)
 |-- EsEHB: string (nullable = true)
 |-- TieneLimiteCuantitativo: string (nullable = true)
 |-- ExcluidoDelDesembolsoMaximoDentroDeLaRed: string (nullable = true)
 |-- ExcluidoDelDesembolsoMaximoFueraDeLaRed: string (nullable = true)
 |-- UnidadDelLimite: string (nullable = true)
 |-- Annio: long (nullable = true)
 |-- IdCondicionesBeneficios_DWH: long (nullable = true)



## Load

In [26]:
mini_condiciones.count()

346

In [27]:
# Se realiza la carga delos datos
conn_dest.save_db(mini_condiciones, 'Rs_MiniCondicionesTipoBeneficio')